In [3]:
# 安装 Graphrag
! pip cache purge
! pip install aiofiles==23.0.0
! pip install graphrag

Files removed: 1180


ERROR: Ignored the following yanked versions: 23.2.0
ERROR: Could not find a version that satisfies the requirement aiofiles==23.0.0 (from versions: 0.2.1, 0.3.0, 0.3.1, 0.3.2, 0.4.0, 0.5.0, 0.6.0, 0.7.0, 0.8.0, 22.1.0, 23.1.0, 23.2.1, 24.1.0)
ERROR: No matching distribution found for aiofiles==23.0.0


In [5]:
# 首次运行需要设置工作区
! python -m graphrag.index --init --root ./graph_data

⠋ GraphRAG Indexer 
Initializing project at ./graph_data
⠋ GraphRAG Indexer 


In [6]:
! mkdir ./graph_data/input

�����﷨����ȷ��


In [16]:
# 建立 rag 索引
! python -m graphrag.index --root ./graph_data

⠋ GraphRAG Indexer 
Logging enabled at 
C:\Users\26645\Desktop\learn\aiops-tutorials\task\week_3\rag\graph_data\output\
indexing-engine.log
⠋ GraphRAG Indexer 
⠋ GraphRAG Indexer 
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
└── create_base_text_units
⠦ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) - 100%  0…
└── create_base_text_units
⠦ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filter

In [13]:
# 检索 graph rag
! python -m graphrag.query --root ./graph_data --method global "谁维护的服务最多"



creating llm client with {'api_key': 'REDACTED,len=51', 'type': "openai_chat", 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://api.apiyi.com/v1', 'api_version': None, 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': None, 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Global Search Response:
根据分析报告，维护服务最多的人员是小张。他负责支付后端和前端操作，这表明他在维护与支付处理相关的服务方面扮演着重要角色。小张的领导对于确保这些系统的顺利功能和效率至关重要，这些系统对任何依赖支付处理的企业都是必不可少的 [Data: Reports (0, 1, 2)]。

其次，小王负责支付网关操作，这对于电子商务中信用卡支付的授权至关重要。他的管理实践对电子商务操作的整体功能性具有重要影响 [Data: Reports (1)]。小李则负责管理支付回调操作，确保商家能够及时收到支付状态通知，这对支付交易的整体效率也有显著影响 [Data: Reports (2)]。

综上所述，小张在维护服务方面的角色最为重要，其次是小王和小李。每个人的职责都对支付处理的不同环节产生了重要影响，确保了整个支付系统的高效运作。


In [14]:
# 启动 Neo4j 查看实体关系
! docker run -d \
    -p 7474:7474 -p 7687:7687 \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4J_PLUGINS=\[\"apoc\"\] \
    neo4j:5.21.2

10bd0f5640a107689fead0801fa19bb19bed91747b2cd68c4c3791e4f6dbe771


Unable to find image 'neo4j:5.21.2' locally
5.21.2: Pulling from library/neo4j
5de87e84afee: Pulling fs layer
5f5c50d2d095: Pulling fs layer
399c5e0f462c: Pulling fs layer
1d820b141c5e: Pulling fs layer
f688f7383f49: Pulling fs layer
1d820b141c5e: Waiting
4f4fb700ef54: Pulling fs layer
f688f7383f49: Waiting
4f4fb700ef54: Waiting
399c5e0f462c: Verifying Checksum
399c5e0f462c: Download complete
1d820b141c5e: Verifying Checksum
1d820b141c5e: Download complete
5de87e84afee: Verifying Checksum
5de87e84afee: Download complete
4f4fb700ef54: Verifying Checksum
4f4fb700ef54: Download complete
5f5c50d2d095: Download complete
5de87e84afee: Pull complete
5f5c50d2d095: Pull complete
399c5e0f462c: Pull complete
1d820b141c5e: Pull complete
f688f7383f49: Download complete
f688f7383f49: Pull complete
4f4fb700ef54: Pull complete
Digest: sha256:fb93fab86f5e0f3e9782ba5bdfbe93578994e244ab114a5c9edc0ce292a0db5f
Status: Downloaded newer image for neo4j:5.21.2


In [15]:
! pip install --quiet pandas neo4j-rust-ext

In [17]:
import pandas as pd
from neo4j import GraphDatabase
import time

# 修改为你的 Graphrag 输出目录
GRAPHRAG_FOLDER = "./graph_data/output/"

NEO4J_URI = "neo4j://localhost"  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "mypassword"
NEO4J_DATABASE = "neo4j"

# Create a Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0,total, batch_size):
        batch = df.iloc[start: min(start+batch_size,total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in { time.time() - start_s} s.')    
    return total

doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_documents.parquet', columns=["id", "title"])
doc_df.head(2)

# import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_text_units.parquet',
                          columns=["id","text","n_tokens","document_ids"])
text_df.head(2)

statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_entities.parquet',
                            columns=["name", "type", "description", "human_readable_id", "id", "description_embedding",
                                     "text_unit_ids"])
entity_df.head(2)

entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_relationships.parquet',
                         columns=["source", "target", "id", "rank", "weight", "human_readable_id", "description",
                                  "text_unit_ids"])
rel_df.head(2)

rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_communities.parquet',
                               columns=["id", "level", "title", "text_unit_ids", "relationship_ids"])

community_df.head(2)

statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_community_reports.parquet',
                                      columns=["id", "community", "level", "title", "summary", "findings", "rank",
                                               "rank_explanation", "full_content"])
community_report_df.head(2)
# import communities
community_statement = """MATCH (c:__Community__ {community: value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id: finding_idx})
SET f += finding"""
batched_import(community_statement, community_report_df)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique
{'_contains_updates': True, 'labels_added': 1, 'nodes_created': 1, 'properties_set': 2}
1 rows in 0.2868232727050781 s.
{'_contains_updates': True, 'labels_added': 2, 'relationships_created': 2, 'nodes_created': 2, 'properties_set': 6}
2 rows in 0.23356294631958008 s.


ClientError: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `apoc.create.addLabels` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}